In [ ]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42


# 26-Class Hand Gesture Recognition Training
This notebook trains a model to recognize all 26 letters of the alphabet (A-Z) using hand gestures.


In [ ]:
# Paths configuration
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier_26.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier_26.tflite'


In [ ]:
# Number of classes (26 letters A-Z)
NUM_CLASSES = 26


In [ ]:
# Load training data
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)


In [ ]:
# Build model for 26 classes
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])


In [ ]:
model.summary()


In [ ]:
# Model callbacks
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)


In [ ]:
# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train model
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)


In [ ]:
# Evaluate model
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)
print(f'Validation Accuracy: {val_acc:.4f}')


In [ ]:
# Load saved model
model = tf.keras.models.load_model(model_save_path)


In [ ]:
# Test prediction
predict_result = model.predict(np.array([X_test[0]]))
print(f'Prediction probabilities: {np.squeeze(predict_result)}')
print(f'Predicted class: {np.argmax(np.squeeze(predict_result))}')


In [ ]:
# Confusion matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    # Create letter labels for display
    letter_labels = [chr(65 + i) for i in labels]  # A=65, B=66, etc.
    
    df_cmx = pd.DataFrame(cmx_data, index=letter_labels, columns=letter_labels)
 
    fig, ax = plt.subplots(figsize=(15, 12))
    sns.heatmap(df_cmx, annot=True, fmt='g', square=False, cmap='Blues')
    ax.set_ylim(len(set(y_true)), 0)
    plt.title('Confusion Matrix - 26 Letter Hand Gesture Recognition')
    plt.xlabel('Predicted Letter')
    plt.ylabel('True Letter')
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred, target_names=letter_labels))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)


In [ ]:
# Convert to TensorFlow Lite
model.save(model_save_path, include_optimizer=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)
print(f'Model saved as {tflite_save_path}')


In [ ]:
# Test TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(f'TFLite prediction: {np.squeeze(tflite_results)}')
print(f'TFLite predicted class: {np.argmax(np.squeeze(tflite_results))}')
